In [1]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR

import models 
import utils

import math
from functools import partial

import torch
from torch import nn
import torch.nn.utils.parametrize as parametrize
import lora 
# import utils

import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, random_split
from torch.utils.data import Dataset, DataLoader, ConcatDataset, Subset
import numpy as np


import torch
import torch.nn as nn
import torch.nn.utils.parametrize as parametrize

In [4]:
import models.resnet


device = 'cuda'
# from models import lenet

model = models.resnet.resnet50(pretrained=False, progress=True).to('cuda')
# model = models.lenet.LeNet().to('cuda')

# model = models.resnet.resnet50(pretrained=True, progress=True, device="cuda").to('cuda')

# layers = [ model.conv1, model.layer1[0].conv1, model.layer1[0].conv2, model.layer1[0].conv3,
#                 model.layer2[0].conv1, model.layer2[0].conv2, model.layer2[0].conv3,
#                 model.layer3[0].conv1, model.layer3[0].conv2, model.layer3[0].conv3,
#                 model.layer4[0].conv1, model.layer4[0].conv2, model.layer4[0].conv3, model.fc ]

# for conv_layer in layers:
#     lora_param = lora.layer_parametrization(conv_layer, device="cuda", rank=10, lora_alpha=1)
#     parametrize.register_parametrization(conv_layer, 'weight', lora_param)

In [3]:
for name, param in model.named_parameters():
    if 'mat' not in name:
        print(f'Freezing non-LoRA parameter {name}')
        param.requires_grad = False

for layer in layers:
  layer.parametrizations["weight"][0].requires_grad = True

for name, param in model.named_parameters():
   if param.requires_grad:
        print(f"Parameter: {name}, Shape: {param.size()}")


Freezing non-LoRA parameter conv1.parametrizations.weight.original
Freezing non-LoRA parameter bn1.weight
Freezing non-LoRA parameter bn1.bias
Freezing non-LoRA parameter layer1.0.conv1.parametrizations.weight.original
Freezing non-LoRA parameter layer1.0.bn1.weight
Freezing non-LoRA parameter layer1.0.bn1.bias
Freezing non-LoRA parameter layer1.0.conv2.parametrizations.weight.original
Freezing non-LoRA parameter layer1.0.bn2.weight
Freezing non-LoRA parameter layer1.0.bn2.bias
Freezing non-LoRA parameter layer1.0.conv3.parametrizations.weight.original
Freezing non-LoRA parameter layer1.0.bn3.weight
Freezing non-LoRA parameter layer1.0.bn3.bias
Freezing non-LoRA parameter layer1.0.downsample.0.weight
Freezing non-LoRA parameter layer1.0.downsample.1.weight
Freezing non-LoRA parameter layer1.0.downsample.1.bias
Freezing non-LoRA parameter layer1.1.conv1.weight
Freezing non-LoRA parameter layer1.1.bn1.weight
Freezing non-LoRA parameter layer1.1.bn1.bias
Freezing non-LoRA parameter layer1

In [5]:
# Setup device
import torch.optim as optim
import active
import trades

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mean_cifar10 = (0.4914, 0.4822, 0.4465)
std_cifar10 = (0.2471, 0.2435, 0.2616)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean_cifar10, std_cifar10)  ])

pool_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
pool_loader = DataLoader(pool_dataset, batch_size=128, shuffle=True)

adapt_dataset = active.EmptyDataset()

pool_indices = list( range(len(pool_loader.dataset ) ) ) 
n_queries = 1
active_batch_size = 10

for i in range(n_queries):

    print( 'iteration {}'.format(i) )
    
    pool_loader = DataLoader( Subset(pool_dataset, pool_indices), batch_size=128, shuffle=False)
    query_indices = active.uncertainty_sampling(model, pool_loader, active_batch_size)

    global_query_indices = [ pool_indices[idx] for idx in query_indices]

    for idx in global_query_indices:
        image, label = pool_dataset[idx]
        adapt_dataset.add_data( [image], [label] )

    # Remove queried instances from the pool
    pool_indices = [idx for idx in pool_indices if idx not in global_query_indices]
    print( len(pool_indices) )

    ################# Update the ResNet through low rank matrices:
    print('start training process')

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    adapt_loader = DataLoader(adapt_dataset, batch_size=128, shuffle=True)
    for _ in range(10):
        print('epoch {}'.format(_) )
        model.train()
        for batch_idx, (data, target) in enumerate(adapt_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            # loss = trades.trades_loss(model=model, x_natural=data, y=target, optimizer=optimizer,)
            loss = nn.CrossEntropyLoss()( model(data) ,target)
            print(loss)
            loss.backward()
            optimizer.step()
            

Files already downloaded and verified
iteration 0


KeyboardInterrupt: 

In [5]:

import numpy as np
import os
import argparse
import os
import torch

from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR

import models 
import utils

import sys

import torch.nn.utils.parametrize as parametrize
import lora 

from torch.utils.data import Subset
import trades
import active
import torch.optim as optim
from torch import nn

import pickle as pkl
import gzip

import random 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mean_cifar10 = (0.4914, 0.4822, 0.4465)
std_cifar10 = (0.2471, 0.2435, 0.2616)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean_cifar10, std_cifar10)  ])


# Uncertainty sampling function
# def uncertainty_sampling(model, loader, n_instances=10):
#     device = 'cuda'
#     model.eval()
#     all_indices = list( range(len(loader.dataset)) )
#     uncertainties = []
    
#     with torch.no_grad():
#         for images, _ in loader:
#             images = images.to(device)
#             outputs = torch.softmax( model(images), dim=1)
#             uncertainty = 1 - torch.max(outputs, dim=1)[0]
#             uncertainties.extend( uncertainty.tolist() )
    
#     # Select the indices of the top uncertain instances
#     uncertainty_indices = np.argsort(uncertainties)[-n_instances:]

#     return [all_indices[i] for i in uncertainty_indices]

def attack_uncertainty_sampling(model, loader, n_instances=10):
    device = 'cuda'
    
    result = []
    epsilon = 8/255
    n_restarts = 2
    attack_iters = 10
    alpha = epsilon/4
    
    for images, labels in loader:
        model.train()
        print(labels.shape)
        images = images.to(device)
        labels = labels.to(device)

        delta = utils.attack_pgd_restart(model, images, labels, epsilon, alpha, attack_iters, n_restarts, rs=True, verbose=False,
                    linf_proj=True, l2_proj=False, l2_grad_update=False, cuda=True)
        
        x_adv = images + delta

        model.eval()

        outputs = torch.softmax( model(images), dim=1)
        uncertainties = 1 - torch.max(outputs, dim=1)[0]

        outputs = model( x_adv )
        _, predicted_labels = torch.max(outputs, dim=1)
        attack_success = predicted_labels != labels

        merge = torch.vstack( [uncertainties, attack_success] )
        print(uncertainties.shape, attack_success.shape, merge.shape)
        
        result.append(merge)

    result = torch.hstack(result).T
    result = result[ result[:, 1] == 1]
    val, idx = result[:, 0].sort(descending=True)
    indices = idx[:n_instances]
    
    return indices


n_rounds = 1

pool_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
pool_loader = DataLoader(pool_dataset, batch_size=128, shuffle=True)

adapt_dataset = active.EmptyDataset()
pool_indices = list( range(250) ) #range(len(pool_loader.dataset ) ) 

epoch_counter = 0

for i in range(n_rounds):

    pool_loader = DataLoader( Subset(pool_dataset, pool_indices), batch_size=50, shuffle=False)

    query_indices = attack_uncertainty_sampling( model, pool_loader, None)
    
    # _ = utils.add_data(query_indices, pool_indices, pool_dataset, adapt_dataset)

    # adapt_loader = DataLoader(adapt_dataset, batch_size=128, shuffle=True)

Files already downloaded and verified
torch.Size([50])
torch.Size([50]) torch.Size([50]) torch.Size([2, 50])
torch.Size([50])
torch.Size([50]) torch.Size([50]) torch.Size([2, 50])
torch.Size([50])
torch.Size([50]) torch.Size([50]) torch.Size([2, 50])
torch.Size([50])
torch.Size([50]) torch.Size([50]) torch.Size([2, 50])
torch.Size([50])
torch.Size([50]) torch.Size([50]) torch.Size([2, 50])


In [15]:
idx

tensor([109, 122,  24,  17,  73,  40, 106, 129, 121,  11,  90, 125,  78,  53,
         64,  23, 123,  54,  85, 105,  39,  29,  42, 120,  55,   8,  45,  63,
        126,  34, 116, 111,  87,  76,  75,  25,  94,  43,   9,  37, 119,  79,
         58,  44,  52,  98,  67,  68,  56,  57,  60,  89,  62,  22,  69,   4,
         48,  92,  31,  66,  95,  13,  28,  12,  49,  27,  26,  51,  30,  21,
        118,  16,  47,  65,  71,   0, 100, 107, 117,  10,  20, 115, 102,  83,
        101,  97,   2,  91,  32,  59,  82,  38,  46,  33, 128,  19,  50,   7,
         74,  18,  84,   6,  86,  77,  35,  15, 108,  61,  72,  36,   1, 124,
         81, 112,  96,   5, 103,  14,  93,  88, 104, 113,  99,  41,  80, 110,
         70, 114,   3, 127], device='cuda:0')